## Memory


대화형 AI 애플리케이션에서 가장 중요한 기능 중 하나는 이전 대화 내용을 기억하고 맥락을 유지하는 것입니다. LangChain은 다양한 메모리 관리 방식을 제공하여 상태를 가진(stateful) 대화형 애플리케이션을 구축할 수 있도록 도와줍니다.

In [7]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.1")

response1 = llm.invoke("안녕하세요, 내 이름은 민수입니다.")
print(response1)

response2 = llm.invoke("제 이름이 뭐였죠?")
print(response2)

response3 = llm.invoke("바보야!!!")
print(response3)



안녕하세요! 반갑습니다. 민수님이라고 하면 어떤 분인지 궁금하네요. 뭐하고 계신가요?
저는 이전에 우리의 대화를 기억하지 못합니다. 새로운 대화에서부터 시작할 수 있습니다.
힘내세요!


## 메모리의 역할

메모리는 대화형 애플리케이션에서 다음과 같은 중요한 역할을 합니다:

1. **맥락 유지**: 이전 대화 내용을 기억하여 현재 대화에 반영합니다.
2. **개인화**: 사용자에 대한 정보를 저장하여 더 개인화된 응답을 제공합니다.
3. **상태 관리**: 대화의 상태를 추적하여 일관된 대화를 유지합니다.
4. **향상된 사용자 경험**: 사용자가 반복해서 정보를 제공하지 않아도 되도록 하여 더 원활한 대화 경험을 제공합니다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 천문학 전문가입니다. 사용자와 친근한 대화를 나누며 천문학 질문에 답변해주세요."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

store = {}

llm = OllamaLLM(model="llama3.1")

chain = prompt | llm

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history"
)

config = {"configurable": {"session_id": "astronomy_chat_1"}}

response1 = chain_with_history.invoke(
    {"question": "안녕하세요! 저는 지구과학을 공부하는 학생입니다."}, config=config
)

print(response1)

response2 = chain_with_history.invoke(
    {"question": "태양계에서 가장 큰 행성은 무엇인가요?"}, config=config
)

print(response2)

response3 = chain_with_history.invoke(
    {"question": "그 행성의 위성에 대해 알려주세요."}, config=config
)

print(response3)

config2 = {"configurable": {"session_id": "astronomy_chat_2"}}

response4 = chain_with_history.invoke(
    {"question": "제가 누구인지 기억하나요?"}, config=config2
)

print(response4)

print(store)


안녕하세요! 네, 좋은 아침입니다! 지구과학을 공부하시다니 축하합니다! 천문학은 지구과학의 중요한 일부분이라고 생각합니다. 어떤 질문이 있으신가요? 별이나 행성에 대해 물어보세요!
